In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install accelerate -U
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install sentencepiece

In [ ]:
import transformers

print(transformers.__version__)

4.38.1


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
model_checkpoint = "vinai/bartpho-syllable"

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("toanduc/10k-final")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
metric = load("rouge")

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Original Text', 'Summary'],
        num_rows: 11585
    })
    validation: Dataset({
        features: ['Original Text', 'Summary'],
        num_rows: 907
    })
})

In [ ]:
raw_datasets["train"][0]

In [ ]:
from transformers import BertTokenizer,BertModel
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = "summary"

In [ ]:
import re
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Original Text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(text_target=examples["Summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/11585 [00:00<?, ? examples/s]

Map:   0%|          | 0/907 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer,MT5ForConditionalGeneration

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
model_name = model_checkpoint.split("/")[-1]
fine_tuned_model_name = f"{model_name}-sport"

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]


    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result = {key: value * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
num_epochs = 16
batch_size = 4
# Thiết lập trainer cho mỗi epoch
args = Seq2SeqTrainingArguments(
  fine_tuned_model_name,
  evaluation_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  weight_decay=0.01,
  save_total_limit=3,
  num_train_epochs=num_epochs,
  predict_with_generate=True,
  fp16=True,
)


trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.728800,1.513201,22.873600,10.779600,17.509500,20.022700,20.000000
2,0.655900,1.578658,22.671900,11.501600,17.621200,20.114600,20.000000
3,0.588000,1.688227,22.523900,11.382100,17.592000,20.000700,20.000000
4,0.499500,1.766729,22.485000,11.034100,17.379000,19.717400,20.000000
5,0.434500,1.883449,22.237400,11.224900,17.435700,19.689700,20.000000
6,0.376000,2.011312,22.006200,10.970100,17.109100,19.440800,20.000000
7,0.332400,2.105233,22.276000,11.074700,17.372900,19.668300,20.000000
8,0.288000,2.202052,22.506100,11.077000,17.324800,19.765200,20.000000
9,0.259700,2.255637,22.124200,10.929700,17.118600,19.520300,20.000000
10,0.214300,2.331360,22.091900,10.897800,17.143300,19.482500,20.000000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

TrainOutput(global_step=46352, training_loss=0.3178191581676037, metrics={'train_runtime': 35328.0726, 'train_samples_per_second': 5.247, 'train_steps_per_second': 1.312, 'total_flos': 2.8024955609264947e+17, 'train_loss': 0.3178191581676037, 'epoch': 16.0})

In [21]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

events.out.tfevents.1710239040.d5f8b6fd30a3.102875.0:   0%|          | 0.00/53.4k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/toanduc/bartpho-syllable-sport/commit/bebca5b304f310e61b670cf5d81262d02b10d561', commit_message='End of training', commit_description='', oid='bebca5b304f310e61b670cf5d81262d02b10d561', pr_url=None, pr_revision=None, pr_num=None)